In [91]:
!pip3 uninstall xlrd

Found existing installation: xlrd 2.0.1
Uninstalling xlrd-2.0.1:
  Would remove:
    /home/personal/.local/bin/runxlrd.py
    /home/personal/.local/lib/python3.8/site-packages/xlrd-2.0.1.dist-info/*
    /home/personal/.local/lib/python3.8/site-packages/xlrd/*
Proceed (y/n)? ^C
ERROR: Operation cancelled by user


In [87]:
import pandas as pd
import numpy as np
import sqlite3
import requests

In [88]:
xls_path = "data/nombres_por_edad_media.xls"

In [92]:
xl = pd.ExcelFile(xls_path)
sheets_lst = xl.sheet_names
print(sheets_lst)

['Hombres', 'Mujeres']


In [93]:
test_name_df = pd.read_excel(
    xls_path, sheet_name = sheets_lst[1], skiprows = 6)

In [94]:
test_name_df.head(5)

,Orden,Nombre,Frecuencia,Edad Media (*)
0,1,MARIA CARMEN,641875,59.5
1,2,MARIA,578124,48.4
2,3,CARMEN,366259,60.5
3,4,ANA MARIA,269814,53.6
4,5,MARIA PILAR,258063,59.4


In [95]:
columns = ["name", "freq", "avg_age"]
test_name_df = test_name_df.drop(columns = ["Orden"])
test_name_df.columns = columns
test_name_df["name"] = test_name_df["name"].str.upper().str.strip()

In [96]:
test_name_df.head()

,name,freq,avg_age
0,MARIA CARMEN,641875,59.5
1,MARIA,578124,48.4
2,CARMEN,366259,60.5
3,ANA MARIA,269814,53.6
4,MARIA PILAR,258063,59.4


In [60]:
def load_and_clean_names_xls(gender_str):
    columns = ["name", "freq", "avg_age"]
    if gender_str == 'M':
        sheet_idx = 0
    elif gender_str == 'F':
        sheet_idx = 1
    else:
        sheet_idx = None
    df = pd.read_excel(
        xls_path, sheet_name = sheets_lst[sheet_idx], skiprows = 6)
    df = df.drop(columns = ["Orden"])
    df.columns = columns
    df["name"] = df["name"].str.upper().str.strip()
    return df

In [61]:
female_name_df = load_and_clean_names_xls("F")
male_name_df = load_and_clean_names_xls("M")

In [62]:
female_name_df.head()

,name,freq,avg_age
0,MARIA CARMEN,641875,59.5
1,MARIA,578124,48.4
2,CARMEN,366259,60.5
3,ANA MARIA,269814,53.6
4,MARIA PILAR,258063,59.4


In [63]:
male_name_df.head()

,name,freq,avg_age
0,ANTONIO,640854,57.1
1,MANUEL,562798,55.7
2,JOSE,547142,62.1
3,FRANCISCO,467933,58.7
4,DAVID,369633,32.9


In [64]:
name_df = pd.merge(
    female_name_df.set_index("name"), 
    male_name_df.set_index("name"), 
    on = ["name"],  
    how = "outer",
    suffixes=('_f', '_m'))

In [65]:
name_df.head()

,freq_f,avg_age_f,freq_m,avg_age_m
name,,,,
MARIA CARMEN,641875.0,59.5,NaN,NaN
MARIA,578124.0,48.4,183.0,55.4
CARMEN,366259.0,60.5,NaN,NaN
ANA MARIA,269814.0,53.6,NaN,NaN
MARIA PILAR,258063.0,59.4,NaN,NaN


In [66]:
name_df["freq_f"] = name_df["freq_f"].fillna(0)
name_df["freq_m"] = name_df["freq_m"].fillna(0)
name_df["prob_f"] = name_df["freq_f"] / (name_df["freq_f"] + name_df["freq_m"])
name_df["prob_m"] = name_df["freq_m"] / (name_df["freq_f"] + name_df["freq_m"])
name_df["gender"] = np.where(name_df["prob_f"] >= 0.5, "F", "M")

In [67]:
name_basics_df = name_df[["prob_f", "prob_m", "gender"]]

In [68]:
name_basics_df.sample(5)

,prob_f,prob_m,gender
name,,,
IVAN LUIS,0.0,1.0,M
LESTER ANTONIO,0.0,1.0,M
STEPHANIE,1.0,0.0,F
JOSSELYN,1.0,0.0,F
DUGUNEH,0.0,1.0,M


In [69]:
name_basics_df.to_csv("data/names_master.csv")

In [70]:
#!pip3 install django djangorestframework markdown django-filter

In [71]:
sqliteConnection = sqlite3.connect('db.sqlite3')
cursor = sqliteConnection.cursor()

sqlite_select_Query = """
    SELECT name
    FROM sqlite_master
    WHERE type ='table';
"""
cursor.execute(sqlite_select_Query)
record = cursor.fetchall()
for r in record:
    print(r)
cursor.close()

('django_migrations',)
('sqlite_sequence',)
('auth_group_permissions',)
('auth_user_groups',)
('auth_user_user_permissions',)
('django_admin_log',)
('django_content_type',)
('auth_permission',)
('auth_group',)
('auth_user',)
('django_session',)
('web_name',)


In [72]:
cursor = sqliteConnection.cursor()
sqlite_select_Query = """
    PRAGMA table_info([web_name]);
"""
cursor.execute(sqlite_select_Query)
record = cursor.fetchall()
for r in record:
    print(r)
cursor.close()

(0, 'id', 'integer', 1, None, 1)
(1, 'text', 'varchar(255)', 1, None, 0)
(2, 'prob_f', 'real', 1, None, 0)
(3, 'prob_m', 'real', 1, None, 0)
(4, 'gender', 'varchar(255)', 1, None, 0)
(5, 'pub_date', 'datetime', 0, None, 0)


In [73]:
cursor = sqliteConnection.cursor()
sqlite_select_Query = """
    SELECT * 
    FROM web_name
    LIMIT 5;
"""
cursor.execute(sqlite_select_Query)
record = cursor.fetchall()
for r in record:
    print(r)
cursor.close()

(36580, 'MARIA CARMEN', 1.0, 0.0, 'F', None)
(36581, 'MARIA', 0.9996835590784825, 0.0003164409215174639, 'F', None)
(36582, 'CARMEN', 1.0, 0.0, 'F', None)
(36583, 'ANA MARIA', 1.0, 0.0, 'F', None)
(36584, 'MARIA PILAR', 1.0, 0.0, 'F', None)


In [84]:
cursor = sqliteConnection.cursor()

for index, row in name_basics_df.reset_index().iterrows():
    name = str(row["name"]).replace("'", " ")  
    prob_f = row["prob_f"]
    prob_m = row["prob_m"]
    gender = row["gender"]    

    insert_into_name = """
        INSERT INTO web_name 
            (text, prob_f, prob_m, gender)
            VALUES ('{0}', {1}, {2}, '{3}');
        """.format(name, prob_f, prob_m, gender) 
    if index == 0:
        print("Example query:")
        print(insert_into_name)
    try:
        cursor.execute(insert_into_name)
    except:
        print(insert_into_name)
    sqliteConnection.commit()  

cursor.close()

Example query:

        INSERT INTO web_name 
            (text, prob_f, prob_m, gender)
            VALUES ('MARIA CARMEN', 1.0, 0.0, 'F');
        


In [83]:
cursor = sqliteConnection.cursor()
sqlite_select_Query = """
    SELECT * 
    FROM web_name
    LIMIT 5;
"""
cursor.execute(sqlite_select_Query)
record = cursor.fetchall()
for r in record:
    print(r)
cursor.close()

In [75]:
response = requests.get("http://127.0.0.1:8000/api/names/GUSTAVO/")

In [76]:
print(response.status_code)

200


In [80]:
print(response.json())

{'text': 'GUSTAVO', 'prob_f': 0.0, 'prob_m': 1.0, 'gender': 'M'}


In [81]:
print(response.json()['text'])

GUSTAVO
